# Document Processing Demo

This notebook demonstrates the document processing pipeline used in the Content Verification Tool:

1. **Docling PDF Conversion** - Converting PDF to DoclingDocument
2. **Docling DOCX Conversion** - Converting DOCX to PDF then DoclingDocument (with LibreOffice)
3. **Hierarchical Pre-chunking** - Using HybridChunker from Docling
4. **Paragraph-level Splitting** - Using LangChain RecursiveCharacterTextSplitter
5. **Sentence-level Splitting** - Using SpaCy sentence detection
6. **Verification Shell Creation** - Creating DocumentChunk objects with page # and item # assignments

## Prerequisites

```bash
# Install dependencies (already in pyproject.toml)
pip install docling docling-core langchain-text-splitters spacy python-docx termcolor

# Download SpaCy model
python -m spacy download en_core_web_sm
```

## Sample Documents

This demo requires sample PDF and DOCX files. You can:
- Use your own legal documents (max 10MB)
- Create sample files named `sample.pdf` and `sample.docx` in the project root
- Update the file paths in the code cells below

## Setup: Import Libraries and Initialize Processors

In [1]:
import os
import sys
from pathlib import Path
from pprint import pprint

# Add backend to path for imports
sys.path.insert(0, str(Path.cwd() / 'backend'))

from app.processing.document_processor import DocumentProcessor
from app.processing.chunker import DocumentChunker
from app.models import ChunkingMode, DocumentChunk

print("=" * 80)
print("INITIALIZING DOCUMENT PROCESSING PIPELINE")
print("=" * 80)

# Initialize processor and chunker
processor = DocumentProcessor()
chunker = DocumentChunker()

print("\n✓ All components initialized successfully")

[CACHE] Initialized cache directory: /tmp/document_cache


/Users/laurentwiesel/Dev/ai-law/content_verification_tool/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[PROCESSOR] Initializing Docling DocumentConverter...
[PROCESSOR] LibreOffice found at: /Applications/LibreOffice.app/Contents/MacOS/soffice
[PROCESSOR] DocumentConverter initialized successfully
[CHUNKER] Initializing chunking strategies...
[CHUNKER] Chunking strategies initialized
[OUTPUT] Initialized output directory: /tmp/output
INITIALIZING DOCUMENT PROCESSING PIPELINE
[PROCESSOR] Initializing Docling DocumentConverter...
[PROCESSOR] LibreOffice found at: /Applications/LibreOffice.app/Contents/MacOS/soffice
[PROCESSOR] DocumentConverter initialized successfully
[CHUNKER] Initializing chunking strategies...
[CHUNKER] Chunking strategies initialized

✓ All components initialized successfully


## Part 1: PDF Processing with Docling

We'll convert a PDF document to a DoclingDocument object, which preserves:
- Page structure and numbers
- Paragraphs and sections
- Tables
- Footnotes
- Provenance data (for tracking text spans across pages)

In [2]:
# Define path to sample PDF
PDF_PATH = "AgentQuality-Abridged.pdf"  # User should replace with their file

print("=" * 80)
print("PART 1: PDF → DoclingDocument")
print("=" * 80)

# Check if file exists
if not Path(PDF_PATH).exists():
    print(f"\n❌ ERROR: File not found: {PDF_PATH}")
    print("Please create a sample PDF file or update PDF_PATH variable")
else:
    # Read file content
    with open(PDF_PATH, "rb") as f:
        pdf_content = f.read()
    
    print(f"\n📄 Processing PDF: {PDF_PATH}")
    print(f"   File size: {len(pdf_content) / 1024:.2f} KB")
    
    # Convert with Docling
    result = processor.convert_document(pdf_content, PDF_PATH, use_cache=True)
    
    docling_doc = result['docling_document']
    
    print(f"\n✓ Conversion successful!")
    print(f"   Filename: {result['filename']}")
    print(f"   Pages: {result['page_count']}")
    print(f"   File size: {result['file_size']} bytes")
    
    # Inspect DoclingDocument structure
    print(f"\n📊 DoclingDocument Structure:")
    print(f"   Type: {type(docling_doc)}")
    print(f"   Has pages: {hasattr(docling_doc, 'pages')}")
    if hasattr(docling_doc, 'pages') and docling_doc.pages:
        print(f"   Total pages: {len(docling_doc.pages)}")
        # Pages is a dict, not a list - show first few keys
        page_keys = list(docling_doc.pages.keys())
        print(f"   Page keys (first 5): {page_keys[:5]}")

2025-11-16 12:58:57,639 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]


PART 1: PDF → DoclingDocument

📄 Processing PDF: AgentQuality-Abridged.pdf
   File size: 134.63 KB
[PROCESSOR] Converting document: AgentQuality-Abridged.pdf
[PROCESSOR] File validation passed: AgentQuality-Abridged.pdf (134.63 KB)
[CACHE] Cache MISS for document d53ac937...
[PROCESSOR] Native PDF detected, processing with OCR disabled
[PROCESSOR] Running Docling conversion on tmpbz4z3_pi.pdf...


2025-11-16 12:58:57,695 - INFO - Going to convert document batch...
2025-11-16 12:58:57,702 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 70256a236a6856c82de2c96fe229a58e
2025-11-16 12:58:57,730 - INFO - Loading plugin 'docling_defaults'
2025-11-16 12:58:57,734 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-16 12:58:57,741 - INFO - Loading plugin 'docling_defaults'
2025-11-16 12:58:57,748 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-16 12:58:57,792 - INFO - Accelerator device: 'mps'
2025-11-16 12:59:00,046 - INFO - Accelerator device: 'mps'
2025-11-16 12:59:00,811 - INFO - Processing document tmpbz4z3_pi.pdf
2025-11-16 12:59:23,206 - INFO - Finished converting document tmpbz4z3_pi.pdf in 25.57 sec.


[PROCESSOR] Conversion successful: 4 pages
[CACHE] Cached document d53ac937...
[PROCESSOR] Cleaned up temporary DOCX file

✓ Conversion successful!
   Filename: AgentQuality-Abridged.pdf
   Pages: 4
   File size: 137865 bytes

📊 DoclingDocument Structure:
   Type: <class 'docling_core.types.doc.document.DoclingDocument'>
   Has pages: True
   Total pages: 4
   Page keys (first 5): [1, 2, 3, 4]


## Part 2: DOCX Processing with LibreOffice + Docling

DOCX files are converted to PDF using LibreOffice first, then processed with Docling.

**Why?**
- Accurate page numbers (DOCX doesn't have fixed pages)
- Consistent processing pipeline (everything goes through PDF)
- Better metadata extraction

In [3]:
# Define path to sample DOCX
DOCX_PATH = "AgentQuality-ShortSummary.docx"  # User should replace with their file

print("=" * 80)
print("PART 2: DOCX → PDF → DoclingDocument")
print("=" * 80)

if not Path(DOCX_PATH).exists():
    print(f"\n❌ ERROR: File not found: {DOCX_PATH}")
    print("Please create a sample DOCX file or update DOCX_PATH variable")
else:
    # Read file content
    with open(DOCX_PATH, "rb") as f:
        docx_content = f.read()
    
    print(f"\n📄 Processing DOCX: {DOCX_PATH}")
    print(f"   File size: {len(docx_content) / 1024:.2f} KB")
    print(f"\n⚠️  Note: This will use LibreOffice for DOCX→PDF conversion")
    
    # Convert with Docling (includes LibreOffice conversion)
    result = processor.convert_document(docx_content, DOCX_PATH, use_cache=True)
    
    docling_doc_docx = result['docling_document']
    
    print(f"\n✓ Conversion successful!")
    print(f"   Original: {result['filename']}")
    print(f"   Pages: {result['page_count']}")
    print(f"   Note: Intermediate PDF was created and cleaned up automatically")

PART 2: DOCX → PDF → DoclingDocument

📄 Processing DOCX: AgentQuality-ShortSummary.docx
   File size: 14.86 KB

⚠️  Note: This will use LibreOffice for DOCX→PDF conversion
[PROCESSOR] Converting document: AgentQuality-ShortSummary.docx
[PROCESSOR] File validation passed: AgentQuality-ShortSummary.docx (14.86 KB)
[CACHE] Cache HIT for document 27d47a85...
[PROCESSOR] Using cached document

✓ Conversion successful!
   Original: AgentQuality-ShortSummary.docx
   Pages: 1
   Note: Intermediate PDF was created and cleaned up automatically


## Part 3: Hierarchical Pre-chunking with HybridChunker

The first processing step uses Docling's **HybridChunker** to:
- Preserve document structure (headings, paragraphs, sections)
- Maintain page numbers and provenance data
- Extract footnotes as separate items
- Handle tables

This creates the foundation for further splitting.

In [4]:
print("=" * 80)
print("PART 3: Hierarchical Pre-chunking")
print("=" * 80)

# Use the PDF document from Part 1
if 'docling_doc' not in locals():
    print("\n❌ ERROR: No DoclingDocument available")
    print("Please run Part 1 first to load a PDF document")
else:
    # Apply hierarchical chunking (internal method)
    base_chunks = chunker._apply_hierarchical_chunking(docling_doc)
    
    print(f"\n✓ Hierarchical chunking complete")
    print(f"   Total base chunks: {len(base_chunks)}")
    
    # Show sample chunks
    print(f"\n📋 Sample Base Chunks (first 5):")
    print("-" * 80)
    
    for i, chunk in enumerate(base_chunks[:5], 1):
        page = chunk['page_number']
        overlap = "⚠️ OVERLAP" if chunk['is_overlap'] else ""
        text_preview = chunk['text'][:100] + "..." if len(chunk['text']) > 100 else chunk['text']
        
        print(f"\n{i}. Page {page} {overlap}")
        print(f"   Text: \"{text_preview}\"")
    
    # Statistics
    pages_with_content = set(chunk['page_number'] for chunk in base_chunks)
    overlap_count = sum(1 for chunk in base_chunks if chunk['is_overlap'])
    
    print(f"\n📊 Statistics:")
    print(f"   Pages with content: {len(pages_with_content)}")
    print(f"   Chunks with overlap: {overlap_count}")
    print(f"   Average chunk length: {sum(len(c['text']) for c in base_chunks) / len(base_chunks):.0f} chars")

PART 3: Hierarchical Pre-chunking
[CHUNKER] Applying HierarchicalChunker...
[CHUNKER] HierarchicalChunker produced 8 chunks

✓ Hierarchical chunking complete
   Total base chunks: 8

📋 Sample Base Chunks (first 5):
--------------------------------------------------------------------------------

1. Page 2 
   Text: "We are at the dawn of the agentic era. The transition from predictable, instruction-based tools to a..."

2. Page 3 
   Text: "- The Trajectory is the Truth: We must evolve beyond evaluating just the final output. The true meas..."

3. Page 3 
   Text: "This guide is structured to build from the " why " to the " what " and finally to the " how ." Use t..."

4. Page 4 
   Text: "- For Product Managers, Data Scientists, and QA Leaders: If you're responsible for what to measure a..."

5. Page 4 
   Text: "- For Team Leads and Strategists: To understand how these pieces create a selfimproving system, read..."

📊 Statistics:
   Pages with content: 4
   Chunks with overlap: 0
   

## Part 4: Paragraph-level Splitting

After hierarchical chunking, we apply **LangChain's RecursiveCharacterTextSplitter** to break content into paragraphs.

**Configuration:**
- chunk_size: 100 characters (configurable)
- chunk_overlap: 10 characters
- Separators: `\n\n`, `\n`, `. `, etc.
- keep_separator: "end" (preserves punctuation)

In [5]:
print("=" * 80)
print("PART 4: Paragraph-level Splitting")
print("=" * 80)

if 'base_chunks' not in locals():
    print("\n❌ ERROR: No base chunks available")
    print("Please run Part 3 first")
else:
    # Apply paragraph splitting
    paragraph_chunks = chunker._apply_paragraph_splitting(base_chunks)
    
    print(f"\n✓ Paragraph splitting complete")
    print(f"   Base chunks: {len(base_chunks)}")
    print(f"   Paragraph chunks: {len(paragraph_chunks)}")
    print(f"   Expansion factor: {len(paragraph_chunks) / len(base_chunks):.2f}x")
    
    # Show sample paragraphs
    print(f"\n📋 Sample Paragraph Chunks (first 5):")
    print("-" * 80)
    
    for i, chunk in enumerate(paragraph_chunks[:5], 1):
        page = chunk['page_number']
        text_preview = chunk['text'][:150] + "..." if len(chunk['text']) > 150 else chunk['text']
        
        print(f"\n{i}. Page {page}")
        print(f"   \"{text_preview}\"")
    
    # Length distribution
    lengths = [len(chunk['text']) for chunk in paragraph_chunks]
    print(f"\n📊 Paragraph Length Statistics:")
    print(f"   Min: {min(lengths)} chars")
    print(f"   Max: {max(lengths)} chars")
    print(f"   Avg: {sum(lengths) / len(lengths):.0f} chars")

PART 4: Paragraph-level Splitting
[CHUNKER] Applying paragraph-level splitting...
[CHUNKER] Paragraph splitting produced 11 chunks

✓ Paragraph splitting complete
   Base chunks: 8
   Paragraph chunks: 11
   Expansion factor: 1.38x

📋 Sample Paragraph Chunks (first 5):
--------------------------------------------------------------------------------

1. Page 2
   "We are at the dawn of the agentic era. The transition from predictable, instruction-based tools to autonomous, goal-oriented AI agents presents one of..."

2. Page 3
   "- The Trajectory is the Truth: We must evolve beyond evaluating just the final output. The true measure of an agent's quality and safety lies in its e..."

3. Page 3
   "This whitepaper is for the architects, engineers, and product leaders building this future. It provides the framework to move from building capable ag..."

4. Page 3
   "This guide is structured to build from the " why " to the " what " and finally to the " how ." Use this section to navigate 

## Part 5: Sentence-level Splitting with SpaCy

For fine-grained verification, we use **SpaCy's sentence boundary detection** to split text into individual sentences.

**Key Features:**
- One sentence per chunk
- Intelligent boundary detection (handles abbreviations, titles, etc.)
- Tracks which base chunk each sentence came from

In [6]:
print("=" * 80)
print("PART 5: Sentence-level Splitting")
print("=" * 80)

if 'base_chunks' not in locals():
    print("\n❌ ERROR: No base chunks available")
    print("Please run Part 3 first")
else:
    # Apply sentence splitting
    sentence_chunks = chunker._apply_sentence_splitting(base_chunks)
    
    print(f"\n✓ Sentence splitting complete")
    print(f"   Base chunks: {len(base_chunks)}")
    print(f"   Sentence chunks: {len(sentence_chunks)}")
    print(f"   Expansion factor: {len(sentence_chunks) / len(base_chunks):.2f}x")
    
    # Show sample sentences
    print(f"\n📋 Sample Sentence Chunks (first 10):")
    print("-" * 80)
    
    for i, chunk in enumerate(sentence_chunks[:10], 1):
        page = chunk['page_number']
        base_idx = chunk.get('base_chunk_index', 'N/A')
        text = chunk['text']
        
        print(f"\n{i}. Page {page} (Base Chunk {base_idx})")
        print(f"   \"{text}\"")
    
    # Sentences per base chunk
    from collections import Counter
    base_chunk_counts = Counter(chunk.get('base_chunk_index', -1) for chunk in sentence_chunks)
    
    print(f"\n📊 Sentence Distribution:")
    print(f"   Total sentences: {len(sentence_chunks)}")
    print(f"   Avg sentences per base chunk: {len(sentence_chunks) / len(base_chunks):.1f}")
    print(f"   Base chunk with most sentences: {max(base_chunk_counts.values())} sentences")

PART 5: Sentence-level Splitting
[CHUNKER] Applying sentence-level splitting with SpaCy...
[CHUNKER] Loading SpaCy model for sentence splitting...
[CHUNKER] SpaCy model ready for sentence splitting
[CHUNKER] Sentence splitting produced 48 individual sentences

✓ Sentence splitting complete
   Base chunks: 8
   Sentence chunks: 48
   Expansion factor: 6.00x

📋 Sample Sentence Chunks (first 10):
--------------------------------------------------------------------------------

1. Page 2 (Base Chunk 0)
   "We are at the dawn of the agentic era."

2. Page 2 (Base Chunk 0)
   "The transition from predictable, instruction-based tools to autonomous, goal-oriented AI agents presents one of the most profound shifts in software engineering in decades."

3. Page 2 (Base Chunk 0)
   "While these agents unlock incredible capabilities, their inherent non-determinism makes them unpredictable and shatters our traditional models of quality assurance."

4. Page 2 (Base Chunk 0)
   "This whitepaper serves

## Part 6: Creating Verification Shells (DocumentChunk Objects)

The final step assigns **item numbers** to each chunk and creates **DocumentChunk** objects ready for verification.

**Item Numbering:**
- **Paragraph mode:** Simple sequential (1, 2, 3...) - resets per page
- **Sentence mode:** Hierarchical (1.1, 1.2, 2.1, 2.2...) - shows base chunk relationship

In [7]:
print("=" * 80)
print("PART 6: Creating DocumentChunk Objects (Paragraph Mode)")
print("=" * 80)

if 'docling_doc' not in locals():
    print("\n❌ ERROR: No DoclingDocument available")
else:
    # Chunk in paragraph mode
    paragraph_doc_chunks = chunker.chunk_document(
        docling_doc, 
        mode=ChunkingMode.PARAGRAPH
    )
    
    print(f"\n✓ Document chunks created")
    print(f"   Total chunks: {len(paragraph_doc_chunks)}")
    print(f"   Mode: {ChunkingMode.PARAGRAPH.value}")
    
    # Show first 10 chunks with full metadata
    print(f"\n📋 Document Chunks (first 10):")
    print("-" * 80)
    
    for chunk in paragraph_doc_chunks[:10]:
        overlap_flag = " [OVERLAP]" if chunk.is_overlap else ""
        text_preview = chunk.text[:80] + "..." if len(chunk.text) > 80 else chunk.text
        
        print(f"\nPage {chunk.page_number}, Item {chunk.item_number}{overlap_flag}")
        print(f"  \"{text_preview}\"")
    
    # Page distribution
    from collections import defaultdict
    chunks_per_page = defaultdict(int)
    for chunk in paragraph_doc_chunks:
        chunks_per_page[chunk.page_number] += 1
    
    print(f"\n📊 Distribution by Page:")
    for page in sorted(chunks_per_page.keys())[:5]:  # First 5 pages
        print(f"   Page {page}: {chunks_per_page[page]} chunks")

PART 6: Creating DocumentChunk Objects (Paragraph Mode)
[CHUNKER] Chunking document in paragraph mode...
[CHUNKER] Applying HierarchicalChunker...
[CHUNKER] HierarchicalChunker produced 8 chunks
[CHUNKER] Applying paragraph-level splitting...
[CHUNKER] Paragraph splitting produced 11 chunks
[CHUNKER] Assigning item numbers (paragraph mode)...
[CHUNKER] Assigned item numbers to 11 chunks
[CHUNKER] Chunking complete: 11 total chunks

✓ Document chunks created
   Total chunks: 11
   Mode: paragraph

📋 Document Chunks (first 10):
--------------------------------------------------------------------------------

Page 2, Item 1
  "We are at the dawn of the agentic era. The transition from predictable, instruct..."

Page 3, Item 1
  "- The Trajectory is the Truth: We must evolve beyond evaluating just the final o..."

Page 3, Item 2
  "This whitepaper is for the architects, engineers, and product leaders building t..."

Page 3, Item 3
  "This guide is structured to build from the " why " to th

In [8]:
print("=" * 80)
print("PART 6b: Creating DocumentChunk Objects (Sentence Mode)")
print("=" * 80)

if 'docling_doc' not in locals():
    print("\n❌ ERROR: No DoclingDocument available")
else:
    # Chunk in sentence mode
    sentence_doc_chunks = chunker.chunk_document(
        docling_doc, 
        mode=ChunkingMode.SENTENCE
    )
    
    print(f"\n✓ Document chunks created")
    print(f"   Total chunks: {len(sentence_doc_chunks)}")
    print(f"   Mode: {ChunkingMode.SENTENCE.value}")
    
    # Show first 15 chunks with hierarchical numbering
    print(f"\n📋 Document Chunks with Hierarchical Numbering (first 15):")
    print("-" * 80)
    
    for chunk in sentence_doc_chunks[:15]:
        overlap_flag = " [OVERLAP]" if chunk.is_overlap else ""
        
        print(f"\nPage {chunk.page_number}, Item {chunk.item_number}{overlap_flag}")
        print(f"  \"{chunk.text}\"")
    
    # Analyze hierarchical structure
    print(f"\n📊 Hierarchical Structure Analysis:")
    
    # Count base chunks (items like 1.x, 2.x, 3.x)
    base_items = set()
    for chunk in sentence_doc_chunks:
        if '.' in chunk.item_number:
            base_items.add(chunk.item_number.split('.')[0])
    
    print(f"   Total sentences: {len(sentence_doc_chunks)}")
    print(f"   Total base chunks (paragraph-level): {len(base_items)}")
    print(f"   Avg sentences per base chunk: {len(sentence_doc_chunks) / len(base_items):.1f}")

PART 6b: Creating DocumentChunk Objects (Sentence Mode)
[CHUNKER] Chunking document in sentence mode...
[CHUNKER] Applying HierarchicalChunker...
[CHUNKER] HierarchicalChunker produced 8 chunks
[CHUNKER] Applying sentence-level splitting with SpaCy...
[CHUNKER] Sentence splitting produced 48 individual sentences
[CHUNKER] Assigning item numbers (sentence mode)...
[CHUNKER] Assigned item numbers to 48 chunks
[CHUNKER] Chunking complete: 48 total chunks

✓ Document chunks created
   Total chunks: 48
   Mode: sentence

📋 Document Chunks with Hierarchical Numbering (first 15):
--------------------------------------------------------------------------------

Page 2, Item 1.1
  "We are at the dawn of the agentic era."

Page 2, Item 1.2
  "The transition from predictable, instruction-based tools to autonomous, goal-oriented AI agents presents one of the most profound shifts in software engineering in decades."

Page 2, Item 1.3
  "While these agents unlock incredible capabilities, their inher

## Part 7: Comparing Chunking Modes

Let's compare the two chunking modes side-by-side to understand their differences.

In [9]:
print("=" * 80)
print("PART 7: Chunking Mode Comparison")
print("=" * 80)

if 'paragraph_doc_chunks' in locals() and 'sentence_doc_chunks' in locals():
    print(f"\n{'Metric':<40} {'Paragraph':<15} {'Sentence':<15}")
    print("-" * 70)
    
    print(f"{'Total chunks':<40} {len(paragraph_doc_chunks):<15} {len(sentence_doc_chunks):<15}")
    
    # Average text length
    para_avg_len = sum(len(c.text) for c in paragraph_doc_chunks) / len(paragraph_doc_chunks)
    sent_avg_len = sum(len(c.text) for c in sentence_doc_chunks) / len(sentence_doc_chunks)
    print(f"{'Avg chunk length (chars)':<40} {para_avg_len:<15.0f} {sent_avg_len:<15.0f}")
    
    # Chunks per page (average)
    para_pages = set(c.page_number for c in paragraph_doc_chunks)
    sent_pages = set(c.page_number for c in sentence_doc_chunks)
    para_per_page = len(paragraph_doc_chunks) / len(para_pages)
    sent_per_page = len(sentence_doc_chunks) / len(sent_pages)
    print(f"{'Avg chunks per page':<40} {para_per_page:<15.1f} {sent_per_page:<15.1f}")
    
    # Overlap counts
    para_overlap = sum(1 for c in paragraph_doc_chunks if c.is_overlap)
    sent_overlap = sum(1 for c in sentence_doc_chunks if c.is_overlap)
    print(f"{'Chunks with overlap flag':<40} {para_overlap:<15} {sent_overlap:<15}")
    
    print(f"\n💡 Recommendations:")
    print(f"   • Use PARAGRAPH mode for: General document verification, faster processing")
    print(f"   • Use SENTENCE mode for: Fine-grained verification, detailed fact-checking")
    
else:
    print("\n❌ Please run Part 6 and 6b first")

PART 7: Chunking Mode Comparison

Metric                                   Paragraph       Sentence       
----------------------------------------------------------------------
Total chunks                             11              48             
Avg chunk length (chars)                 486             111            
Avg chunks per page                      2.8             12.0           
Chunks with overlap flag                 0               0              

💡 Recommendations:
   • Use PARAGRAPH mode for: General document verification, faster processing
   • Use SENTENCE mode for: Fine-grained verification, detailed fact-checking


## Part 8: Exporting Verification Shell

Now that we have DocumentChunk objects, let's see how they would be exported for verification.

In [10]:
import json

print("=" * 80)
print("PART 8: Exporting DocumentChunks to JSON")
print("=" * 80)

if 'paragraph_doc_chunks' in locals():
    # Convert to JSON-serializable format
    chunks_data = [
        {
            "page_number": chunk.page_number,
            "item_number": chunk.item_number,
            "text": chunk.text,
            "is_overlap": chunk.is_overlap,
            "verified": None,  # To be filled by AI
            "verification_score": None,
            "verification_source": "",
            "verification_note": ""
        }
        for chunk in paragraph_doc_chunks[:5]  # First 5 for demo
    ]
    
    # Pretty print JSON
    print("\n📄 Sample Export (first 5 chunks):")
    print(json.dumps(chunks_data, indent=2, ensure_ascii=False))
    
    # Save to file
    output_file = "verification_shell.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(
            [chunk.model_dump() for chunk in paragraph_doc_chunks], 
            f, 
            indent=2, 
            ensure_ascii=False
        )
    
    print(f"\n✓ Exported {len(paragraph_doc_chunks)} chunks to: {output_file}")
    
else:
    print("\n❌ No chunks available for export")

PART 8: Exporting DocumentChunks to JSON

📄 Sample Export (first 5 chunks):
[
  {
    "page_number": 2,
    "item_number": "1",
    "text": "We are at the dawn of the agentic era. The transition from predictable, instruction-based tools to autonomous, goal-oriented AI agents presents one of the most profound shifts in software engineering in decades. While these agents unlock incredible capabilities, their inherent non-determinism makes them unpredictable and shatters our traditional models of quality assurance.\nThis whitepaper serves as a practical guide to this new reality, founded on a simple but radical principle:\nAgent quality is an architectural pillar, not a final testing phase.\nThis guide is built on three core messages:",
    "is_overlap": false,
    "verified": null,
    "verification_score": null,
    "verification_source": "",
    "verification_note": ""
  },
  {
    "page_number": 3,
    "item_number": "1",
    "text": "- The Trajectory is the Truth: We must evolve beyo

## Summary

This notebook demonstrated the complete document processing pipeline:

1. ✅ **PDF Conversion** - Docling converts PDF to structured DoclingDocument
2. ✅ **DOCX Conversion** - LibreOffice → PDF → DoclingDocument for accurate pagination
3. ✅ **Hierarchical Chunking** - HybridChunker preserves document structure
4. ✅ **Paragraph Splitting** - RecursiveCharacterTextSplitter for paragraph-level chunks
5. ✅ **Sentence Splitting** - SpaCy sentencizer for fine-grained chunks
6. ✅ **DocumentChunk Creation** - Structured objects with page/item metadata

### Key Takeaways

- **Docling** provides robust PDF/DOCX parsing with metadata preservation
- **HybridChunker** maintains document hierarchy while chunking
- **Paragraph mode** creates ~100-char chunks with simple numbering (1, 2, 3...)
- **Sentence mode** creates true sentence-level chunks with hierarchical numbering (1.1, 1.2...)
- **DocumentChunk objects** are ready for AI verification with Gemini

### Next Steps

- Explore `gemini_features.ipynb` to see AI verification in action
- Run the full application: `./start_all.sh`
- Check `backend/app/processing/` for implementation details
- Try different documents and chunking modes to understand behavior

In [ ]:
# # Restart the kernel to reload the updated document_processor module
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 